##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Um tutorial sobre Multi-Armed Bandits com características por braço

### Como começar

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/per_arm_bandits_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/per_arm_bandits_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/per_arm_bandits_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este tutorial é um guia passo a passo sobre como usar a biblioteca TF-Agents para problemas de bandits contextuais em que as ações (braços) têm características próprias, como uma lista de filmes representados por características (gênero, ano de lançamento...).

### Pré-requisito

Presume-se que o leitor já esteja um pouco familiarizado com a biblioteca Bandit do TF-Agents, principalmente, que tenha trabalhado com o [tutorial para Bandits no TF-Agents](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb) antes de ler este tutorial.

## Multi-Armed Bandits com características de braço

No "clássico" contexto de Contextual Multi-Armed Bandits, um agente recebe um vetor de contexto (ou seja, uma observação) a cada timestep e precisa escolher entre um conjunto finito de ações numeradas (braços) para maximizar a recompensa cumulativa.

Agora, considere o cenário em que um agente recomenda a um usuário o próximo filme para assistir. Sempre que uma decisão precisa ser feita, o agente recebe como contexto algumas informações sobre o usuário (histórico de visualização, preferência de gênero etc...), além da lista de filmes que podem ser escolhidos.

Podemos tentar formular esse problema ao ter as informações do usuário como o contexto, e os braços seriam `movie_1, movie_2, ..., movie_K`, mas essa abordagem tem vários defeitos:

- O número de ações precisaria ser todos os filmes no sistema, e adicionar um novo filme é complicado.
- O agente precisa aprender um modelo para cada filme.
- A semelhança entre filmes não é considerada.

Em vez de numerar os filmes, podemos fazer algo mais intuitivo: representar filmes com um conjunto de características, incluindo gênero, duração, elenco, classificação, ano etc. Essa abordagem tem inúmeras vantagens:

- Generalização dos filmes.
- O agente aprende só uma função de recompensa que os modelos recompensam com as características de usuário e filme.
- É fácil remover ou introduzir novos filmes no sistema.

Nesse novo cenário, o número de ações nem precisa ser o mesmo a cada timestep.


## Bandits por braço no TF-Agents

A suíte Bandit do TF-Agents foi criada para que ninguém possa usá-la para o caso por braço (per-arm) também. Há ambientes por braço, e a maioria das políticas e dos agentes podem operar no modo por braço.

Antes de começar a programar um exemplo, precisamos das importações necessárias.

### Instalação

In [ ]:
!pip install tf-agents

### Importação

In [ ]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

nest = tf.nest

### Parâmetros: fique à vontade para experimentar

In [ ]:
# The dimension of the global features.
GLOBAL_DIM = 40  #@param {type:"integer"}
# The elements of the global feature will be integers in [-GLOBAL_BOUND, GLOBAL_BOUND).
GLOBAL_BOUND = 10  #@param {type:"integer"}
# The dimension of the per-arm features.
PER_ARM_DIM = 50  #@param {type:"integer"}
# The elements of the PER-ARM feature will be integers in [-PER_ARM_BOUND, PER_ARM_BOUND).
PER_ARM_BOUND = 6  #@param {type:"integer"}
# The variance of the Gaussian distribution that generates the rewards.
VARIANCE = 100.0  #@param {type: "number"}
# The elements of the linear reward parameter will be integers in [-PARAM_BOUND, PARAM_BOUND).
PARAM_BOUND = 10  #@param {type: "integer"}

NUM_ACTIONS = 70  #@param {type:"integer"}
BATCH_SIZE = 20  #@param {type:"integer"}

# Parameter for linear reward function acting on the
# concatenation of global and per-arm features.
reward_param = list(np.random.randint(
      -PARAM_BOUND, PARAM_BOUND, [GLOBAL_DIM + PER_ARM_DIM]))

### Um ambiente simples por braço

O ambiente estocástico estacionário, explicado no outro [tutorial](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb), tem um equivalente por braço.

Para inicializar o ambiente por braço, é preciso definir as funções que geram

- *características globais e por braço*: essas funções não têm parâmetros de entrada e geram um único vetor de característica (global ou por braço) quando chamadas.
- *recompensas*: essa função aceita como parâmetro a concatenação de um vetor de característica global e por braço e gera uma recompensa. Basicamente, essa é a função que o agente precisará "adivinhar". É importante notar aqui que no caso por braço a função de recompensa é idêntica para todos os braços. Essa é uma diferença fundamental do caso clássico de bandit, em que o agente precisa estimar as funções de recompensa para cada braço de maneira independente.


In [ ]:
def global_context_sampling_fn():
  """This function generates a single global observation vector."""
  return np.random.randint(
      -GLOBAL_BOUND, GLOBAL_BOUND, [GLOBAL_DIM]).astype(np.float32)

def per_arm_context_sampling_fn():
  """"This function generates a single per-arm observation vector."""
  return np.random.randint(
      -PER_ARM_BOUND, PER_ARM_BOUND, [PER_ARM_DIM]).astype(np.float32)

def linear_normal_reward_fn(x):
  """This function generates a reward from the concatenated global and per-arm observations."""
  mu = np.dot(x, reward_param)
  return np.random.normal(mu, VARIANCE)

Agora estamos preparados para inicializar nosso ambiente.

In [ ]:
per_arm_py_env = p_a_env.StationaryStochasticPerArmPyEnvironment(
    global_context_sampling_fn,
    per_arm_context_sampling_fn,
    NUM_ACTIONS,
    linear_normal_reward_fn,
    batch_size=BATCH_SIZE
)
per_arm_tf_env = tf_py_environment.TFPyEnvironment(per_arm_py_env)

Abaixo, podemos verificar o que esse ambiente produz.

In [ ]:
print('observation spec: ', per_arm_tf_env.observation_spec())
print('\nAn observation: ', per_arm_tf_env.reset().observation)

action = tf.zeros(BATCH_SIZE, dtype=tf.int32)
time_step = per_arm_tf_env.step(action)
print('\nRewards after taking an action: ', time_step.reward)

Vemos que a especificação da observação é um dicionário com dois elementos:

- Um com a chave `'global'`: essa é a parte de contexto do global, com o formato que corresponde ao parâmetro `GLOBAL_DIM`.
- Um com a chave `'per_arm'`: é o contexto por braço, e o formato é `[NUM_ACTIONS, PER_ARM_DIM]`. Essa parte é o marcador de posição para as características de braço para cada braço em um timestep.


### Agente LinUCB

O agente LinUCB implementa o algoritmo Bandit de mesmo nome, que estima o parâmetro da função de recompensa linear enquanto mantém um elipsoide de confiança perto da estimativa. O agente escolhe o braço com a recompensa esperada de estimativa mais alta, presumindo que o parâmetro está no elipsoide de confiança.

A criação de um agente requer o conhecimento da especificação da observação e da ação. Ao definir o agente, configuramos o parâmetro booleano `accepts_per_arm_features` como `True`.

In [ ]:
observation_spec = per_arm_tf_env.observation_spec()
time_step_spec = ts.time_step_spec(observation_spec)
action_spec = tensor_spec.BoundedTensorSpec(
    dtype=tf.int32, shape=(), minimum=0, maximum=NUM_ACTIONS - 1)

agent = lin_ucb_agent.LinearUCBAgent(time_step_spec=time_step_spec,
                                     action_spec=action_spec,
                                     accepts_per_arm_features=True)

### Fluxo dos dados de treinamento

Esta seção mostra um pouco como as características por braço saem da política para o treinamento. Fique à vontade para pular para a próxima seção (Definição da métrica de arrependimento) e voltar mais tarde se tiver interesse.

Primeiro, vamos conferir a especificação dos dados no agente. O atributo `training_data_spec` do agente especifica quais elementos e estrutura os dados de treinamento devem ter.

In [ ]:
print('training data spec: ', agent.training_data_spec)

Se observamos a parte `observation` da especificação com mais atenção, notamos que ela não contém características por braço!

In [ ]:
print('observation spec in training: ', agent.training_data_spec.observation)

O que aconteceu com as características por braço? Para responder a essa pergunta, primeiro notamos o seguinte: quando o agente LinUCB treina, ele não precisa das características por braço de **todos** os braços, só daquelas do braço **escolhido**. Portanto, faz sentido abandonar o tensor de formato `[BATCH_SIZE, NUM_ACTIONS, PER_ARM_DIM]`, já que é bastante ineficaz, especialmente se o número de ações for grande.

Ainda assim, as características do braço escolhido precisam estar em algum lugar! Para isso, precisamos conferir se a política do LinUCB armazena as características do braço escolhido no campo `policy_info` dos dados de treinamento:

In [ ]:
print('chosen arm features: ', agent.training_data_spec.policy_info.chosen_arm_features)

Vimos com o formato que o campo `chosen_arm_features` só tem o vetor de característica de um braço, e esse será o braço escolhido. Observe que a `policy_info` (e, com isso, as `chosen_arm_features`) faz parte dos dados de treinamento, como vimos ao inspecionar a especificação dos dados de treinamento, e está disponível no momento do treinamento.

### Definição da métrica de arrependimento

Antes de iniciar o loop de treinamento, definimos algumas funções utilitárias que ajudam a calcular o arrependimento do nosso agente. Essas funções ajudam a determinar a melhor recompensa esperada considerando o conjunto de ações (dado pelas características do braço) e o parâmetro linear que é oculto do agente.

In [ ]:
def _all_rewards(observation, hidden_param):
  """Outputs rewards for all actions, given an observation."""
  hidden_param = tf.cast(hidden_param, dtype=tf.float32)
  global_obs = observation['global']
  per_arm_obs = observation['per_arm']
  num_actions = tf.shape(per_arm_obs)[1]
  tiled_global = tf.tile(
      tf.expand_dims(global_obs, axis=1), [1, num_actions, 1])
  concatenated = tf.concat([tiled_global, per_arm_obs], axis=-1)
  rewards = tf.linalg.matvec(concatenated, hidden_param)
  return rewards

def optimal_reward(observation):
  """Outputs the maximum expected reward for every element in the batch."""
  return tf.reduce_max(_all_rewards(observation, reward_param), axis=1)

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward)

Agora está tudo pronto para iniciar o loop de treinamento do bandit. O driver abaixo escolhe as ações usando a política, armazena as recompensas das ações escolhidas no buffer de replay, calcula a métrica de arrependimento predefinida e executa o passo de treinamento do agente.

In [ ]:
num_iterations = 20 # @param
steps_per_loop = 1 # @param

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.policy.trajectory_spec,
    batch_size=BATCH_SIZE,
    max_length=steps_per_loop)

observers = [replay_buffer.add_batch, regret_metric]

driver = dynamic_step_driver.DynamicStepDriver(
    env=per_arm_tf_env,
    policy=agent.collect_policy,
    num_steps=steps_per_loop * BATCH_SIZE,
    observers=observers)

regret_values = []

for _ in range(num_iterations):
  driver.run()
  loss_info = agent.train(replay_buffer.gather_all())
  replay_buffer.clear()
  regret_values.append(regret_metric.result())


Agora vamos ver o resultado. Se fizermos tudo certo, o agente também conseguirá estimar a função de recompensa linear e, portanto, a política poderá escolher as ações com uma recompensa esperada perto da ideal. Isso é indicado pela nossa métrica de arrependimento definida acima, que diminui e se aproxima de zero.

In [ ]:
plt.plot(regret_values)
plt.title('Regret of LinUCB on the Linear per-arm environment')
plt.xlabel('Number of Iterations')
_ = plt.ylabel('Average Regret')

### Próximos passos

O exemplo acima é [implementado](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/examples/v2/train_eval_per_arm_stationary_linear.py) na nossa base de código, onde você pode escolher entre outros agentes também, incluindo o [agente Neural epsilon-Greedy](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/neural_epsilon_greedy_agent.py).